In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

In [ ]:
def gompertz(t,m,A,l):
    return A * np.exp(-np.exp(m*np.e/A*(l-t) + 1))

In [ ]:
def add_subplot_axes(ax,rect,axisbg='w'):
    fig = plt.gcf()
    box = ax.get_position()
    width = box.width
    height = box.height
    inax_position  = ax.transAxes.transform(rect[0:2])
    transFigure = fig.transFigure.inverted()
    infig_position = transFigure.transform(inax_position)    
    x = infig_position[0]
    y = infig_position[1]
    width *= rect[2]
    height *= rect[3]  # <= Typo was here
    subax = fig.add_axes([x,y,width,height],axisbg=axisbg)
    x_labelsize = subax.get_xticklabels()[0].get_size()
    y_labelsize = subax.get_yticklabels()[0].get_size()
    x_labelsize *= rect[2]**0.5
    y_labelsize *= rect[3]**0.5
    subax.xaxis.set_tick_params(labelsize=x_labelsize)
    subax.yaxis.set_tick_params(labelsize=y_labelsize)
    return subax


In [ ]:
def resid(m,y):
    beta = np.dot(np.linalg.pinv(np.dot(m.T,m)), np.dot(m.T, y.T))
    return (y.T-np.dot(m,beta)).T

In [ ]:
def buildCov(x, k1, k2):
    cov = np.zeros((x.shape[0]*4, x.shape[0]*4))

    cov[x.shape[0]:2*x.shape[0],x.shape[0]:2*x.shape[0]] += k1.K(x)
    cov[x.shape[0]:2*x.shape[0],3*x.shape[0]:] += k1.K(x)
    cov[3*x.shape[0]:,x.shape[0]:2*x.shape[0]] += k1.K(x)

    cov[2*x.shape[0]:3*x.shape[0],2*x.shape[0]:3*x.shape[0]] += k2.K(x)
    cov[2*x.shape[0]:3*x.shape[0],3*x.shape[0]:] += k2.K(x)
    cov[3*x.shape[0]:,2*x.shape[0]:3*x.shape[0]] += k2.K(x)

    cov[3*x.shape[0]:,3*x.shape[0]:] += kbatch.K(x) + krep.K(x)

    # equal variance
    #cov[range(cov.shape[0]),range(cov.shape[0])] = np.diag(cov).max()
    
    return cov

In [ ]:
# kbase = GPy.kern.RBF(1,name='base')
# kbio = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1)])
# ktech = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1), GPy.kern.RBF(1,name='tech', variance=.05)])

In [ ]:
nbatch = 3
nrep = 4
ntot = nbatch * nrep
nobs = 30
p = 2

x = np.zeros((nobs*ntot,p+2))
x[:,0] = np.tile(np.linspace(0,2, nobs), ntot)
x[:,1] = np.tile(np.repeat([0,1], nobs), ntot/2)

In [ ]:
nbatch = 3
nrep = 4
ntot = nbatch * nrep
nobs = 30
p = 2

x = np.zeros((nobs*ntot,p+2))
x1 = np.zeros((nobs*ntot,p+1))
x2 = np.zeros((nobs*ntot,p+1))

x[:,0] = x1[:,0] = x2[:,0] = np.tile(np.linspace(0,2, nobs), ntot)
x[:,1] = x1[:,1] = x2[:,1] = np.tile(np.repeat([0,1], nobs), ntot/2)
x[:,2] = x2[:,2] = np.repeat(np.arange(nbatch), nrep*nobs)
x[:,3] = x1[:,2] = np.repeat(np.arange(nbatch*nrep), nobs)

x3 = x.copy()

In [ ]:
baseVariance, batchVariance, repVariance = .01, .01, .01

kbase = GPy.kern.RBF(2,name='base',variance=baseVariance)
kbatch = GPy.kern.IndependentOutputs(GPy.kern.RBF(2,name='batch', variance=batchVariance), index_dim=-2);
krep = GPy.kern.IndependentOutputs(GPy.kern.RBF(2,name='replicate', variance=repVariance))

In [ ]:
plt.imshow(kbatch.K(x))

plt.figure()
plt.imshow(krep.K(x))

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(221)
plt.imshow(kbase.K(x))

plt.subplot(222)
plt.imshow(kbatch.K(x) + kbase.K(x))

plt.subplot(223)
plt.imshow(krep.K(x) + kbase.K(x))

plt.subplot(224)
plt.imshow(krep.K(x) + kbatch.K(x) + kbase.K(x))

In [ ]:
def generateSample(mu, cov, nugget, length=50):
    noise = np.eye(mu.shape[0])*nugget
    
    return scipy.stats.multivariate_normal.rvs(mu,cov+noise).reshape((mu.shape[0]/length,length)).T

In [ ]:
cov = np.zeros((x.shape[0]*4, x.shape[0]*4))

# cov = kbase.K(np.tile(x.T,4).T) #+ batchVariance + repVariance

print cov.shape

# cov[:,:x.shape[0]] = np.tile(kbase.K(x), 4).T
# cov[:x.shape[0],:] = np.tile(kbase.K(x), 4)
# cov[:x.shape[0],:] = kbase.K(x)
cov[x.shape[0]:2*x.shape[0],x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance
cov[x.shape[0]:2*x.shape[0],3*x.shape[0]:] += kbatch.K(x) #- batchVarianceA
cov[3*x.shape[0]:,x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance

cov[2*x.shape[0]:3*x.shape[0],2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance
cov[2*x.shape[0]:3*x.shape[0],3*x.shape[0]:] += krep.K(x) #- repVariance
cov[3*x.shape[0]:,2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance

cov[3*x.shape[0]:,3*x.shape[0]:] += kbatch.K(x) + krep.K(x) # - repVariance-  batchVariance

# equal variance
#cov[range(cov.shape[0]),range(cov.shape[0])] = np.diag(cov).max()

plt.imshow(cov)

In [ ]:
np.random.seed(3)

sigma = .001
f = gompertz(x[:,0], 2, 1, .4)
s = generateSample(np.tile(f, 4), cov, sigma,nobs)

In [ ]:
plt.plot(s);

In [ ]:
plt.figure(figsize=(6,6))
for i in range(4):
    ax = plt.subplot(2,2,[0,2,1,3][i]+1)
    
    plt.title('$M_%d$'%[0,2,1,3][i])
    
    if i % 2 == 1:
        plt.xlabel("time (AU)",)
    else:
        plt.xticks([0,20,40],['']*3)
        
    if i < 2:
        plt.ylabel("growth (AU)")
    else:
        plt.yticks(np.arange(-.2,1.4,.2), ['']*8)
    plt.plot(x[:nobs,0],f[:nobs],c='k', lw=3)
    
    for j,z in enumerate(x[::nobs,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(x[:nobs,0],s[:,ntot*i:ntot*(i+1)][:,j],color='C%d'%k,alpha=.6);
            
    plt.ylim(-.24, 1.29)
    
    subpos = [.45,.05,.5,.4]
    a = add_subplot_axes(ax,subpos)
    #n, bins, patches = plt.hist(x[:,0], 400, normed=1)
    plt.plot([x[:,0].min(), x[:,0].max()], [0,0], c='k', lw=3)
    for j,z in enumerate(x[::nobs,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(x[:nobs,0],s[:,ntot*i:ntot*(i+1)][:,j]-f[:nobs],color='C%d'%k, alpha=.5);
    plt.xticks([])
    plt.yticks([])
            
#plt.tight_layout()
plt.savefig("figures/simulated-data.pdf", bbox_inches='tight')

In [ ]:
def autocorr(x, t=1):
    return numpy.corrcoef(numpy.array([x[0:len(x)-t], x[t:len(x)]]))[0,1]

In [ ]:
cor = np.apply_along_axis(lambda x: [autocorr(x-f[:nobs], z) for z in range(1, nobs-2)], 0, s)

In [ ]:
res = []
for _ in range(100):
    s = generateSample(np.tile(f, 4), cov, sigma,nobs)
    y0, y2 , y1, y3 = np.array_split(s,4,1)
    res.append([-np.log10(scipy.stats.f_oneway(*np.split(resid(m0,y), ntot, 1)).pvalue[0]) for y in [y0, y1, y2, y3]])
res = np.array(res)

In [ ]:
plt.figure(figsize=(12,9))
for i in range(4):
    plt.subplot2grid((4,4),(0,[0,2,1,3][i]))
    plt.title('$M_%d$'%[0,2,1,3][i])
    
    plt.plot(x[:nobs,0],f[:nobs],c='k', lw=3)
    for j,z in enumerate(x[::nobs,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(x[:nobs,0],s[:,ntot*i:ntot*(i+1)][:,j],color='C%d'%k,alpha=.6);
            
    plt.ylim(-.4, 1.29)
    
    plt.subplot2grid((4,4),(1,[0,2,1,3][i]))
    plt.plot([x[:,0].min(), x[:,0].max()], [0,0], c='k', lw=3)
    for j,z in enumerate(x[::nobs,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(x[:nobs,0],s[:,ntot*i:ntot*(i+1)][:,j]-f[:nobs],color='C%d'%k, alpha=.5);
    #plt.xticks([])
    #plt.yticks([])
    
    plt.subplot2grid((4,4),(2,[0,2,1,3][i]))
    plt.boxplot(cor[:,ntot*i:ntot*(i+1)].T);
    plt.xticks([])

plt.subplot2grid((4,4),(3,0),colspan=4)
plt.plot([0,5],[-np.log10(.05)]*2)
# plt.boxplot(res, labels=["$M_%d$"%i for i in range(4)]);
plt.boxplot(res, labels=["" for i in range(4)]);
            
plt.tight_layout()
plt.savefig("figures/simulated-data-alt.pdf", bbox_inches='tight')

In [ ]:
cor = np.apply_along_axis(lambda x: [autocorr(x-f[:nobs], z) for z in range(1, nobs-2)], 0, s)

plt.figure(figsize=(10,10))
for i in range(4):
    ax = plt.subplot(2,2,[0,2,1,3][i]+1)
    
    plt.title('$M_%d$'%[0,2,1,3][i])
    plt.boxplot(cor[:,ntot*i:ntot*(i+1)].T);
    plt.xticks([])
    
plt.savefig("figures/simulated-residuals-autocor.pdf",bbox_inches='tight')

In [ ]:
import patsy

In [ ]:
h = patsy.contrasts.Helmert()

In [ ]:
c = h.code_without_intercept(range(10))

In [ ]:
c.matrix

In [ ]:
kbatch = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='batch', variance=.5), index_dim=-2);
krep = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='replicate', variance=.1))

cov = buildCov(x, kbatch, krep)
plt.imshow(cov)

In [ ]:
s = generateSample(np.tile(f, 4), cov, sigma,nobs)
y0, y2 , y1, y3 = np.array_split(s,4,1)

y0.shape

In [ ]:
plt.subplot(221)
plt.plot(y0)

plt.subplot(222)
plt.plot(y1)

plt.subplot(223)
plt.plot(y2)

plt.subplot(224)
plt.plot(y3)

In [ ]:
m0 = np.ones((ntot,1))

h1 = patsy.contrasts.Helmert().code_without_intercept(range(ntot)).matrix

m1 = np.zeros((ntot,1+h1.shape[1]))
m1[:,0] = 1
m1[:,1:] = h1

h2 = patsy.contrasts.Helmert().code_without_intercept(range(batch)).matrix
m2 = np.zeros((ntot,1+nbatch))
m2[:,0] = 1
for i in range(nbatch):
    m2[i*nrep:(i+1)*nrep,1+i] = 1
    

m3 = np.zeros((ntot,1+nbatch+ntot))
m3[:,0] = 1
for i in range(nbatch):
    m3[i*nrep:(i+1)*nrep,1+i] = 1
    m3[:,1+nbatch:] = np.eye(ntot)
    
plt.imshow(m3)

In [ ]:
for m in [m0, m1, m2, m3]:
    
    plt.figure(figsize=(6,6))

    plt.subplot(221)
    plt.plot(resid(m,y0));

    plt.subplot(222)
    plt.plot(resid(m,y1));

    plt.subplot(223)
    plt.plot(resid(m,y2));

    plt.subplot(224)
    plt.plot(resid(m,y3));

In [ ]:

plt.bar(range(4),
        [-np.log10(scipy.stats.f_oneway(*np.split(resid(m0,y), ntot, 1)).pvalue) for y in [y0, y1, y2, y3]],
        tick_label=['y0','y1','y2','y3'])

In [ ]:
[-np.log10(scipy.stats.f_oneway(*np.split(resid(m0,y), ntot, 1)).pvalue[0]) for y in [y0, y1, y2, y3]],

In [ ]:
sigma, f

In [ ]:
res = []
for _ in range(500):
    s = generateSample(np.tile(f, 4), cov, sigma,nobs)
    y0, y2 , y1, y3 = np.array_split(s,4,1)
    res.append([-np.log10(scipy.stats.f_oneway(*np.split(resid(m0,y), ntot, 1)).pvalue[0]) for y in [y0, y1, y2, y3]])
res = np.array(res)

In [ ]:
plt.plot([0,5],[-np.log10(.05)]*2)
plt.boxplot(res, labels=range(4));

In [ ]:
h.shape

In [ ]:
plt.imshow(m1)

In [ ]:
m1.shape

In [ ]:
m = m2
y = y1
beta = np.dot(np.linalg.pinv(np.dot(m.T,m)), np.dot(m.T, y.T))

plt.plot(beta.T);

In [ ]:
plt.plot(np.dot(m, beta).T,c='k');
plt.plot(y, c='r');

In [ ]:
plt.plot((y.T-np.dot(m,beta)).T)

In [ ]:
beta.shape

In [ ]:
plt.figure(figsize=(10,10))
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.plot([0, 50], [0,0], c='k', lw=3)
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(s[:,ntot*i:ntot*(i+1)][:,j]-f[:50],color='C%d'%k);

In [ ]:
s.shape, ntot

In [ ]:
nsamp = 3
scores = []

for _ in range(nsamp):
    

In [ ]:
baseVariance, batchVariance, repVariance = .01, .01, .005

kbase = GPy.kern.RBF(1,name='base',variance=baseVariance)
kbatch = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='batch', variance=batchVariance), index_dim=-2);
krep = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='replicate', variance=repVariance))

k0 = kbase
k1 = kbase + krep
k2 = kbase + kbatch
k3 = kbase + kbatch + krep

In [ ]:
class OtherHierarchy(GPy.kern.Hierarchical):
    def __init__(self, kernels, extra_dims=None, name='hierarchy'):
        assert all([k.input_dim==kernels[0].input_dim for k in kernels])
        assert len(kernels) > 1
        self.levels = len(kernels) -1
        input_max = max([k.input_dim for k in kernels])
        
        if extra_dims is None:
            extra_dims = range(input_max, input_max + len(kernels)-1)
        
        GPy.kern.src.kern.CombinationKernel.__init__(self,kernels=kernels, extra_dims=extra_dims, name=name)

In [ ]:
m = GPy.models.GPRegression(x[:,[0]], y.T.reshape(50*ntot,1))
m.randomize()
m.optimize()
m

In [ ]:
ax = m.plot_f()
ax.plot(x[:50,0], f[:50],lw=1,color='r')
plt.ylim(0,1)

In [ ]:
plt.imshow(cov)
plt.colorbar()

In [ ]:
cov = np.zeros((x.shape[0]*4, x.shape[0]*4))

# cov = kbase.K(np.tile(x.T,4).T) #+ batchVariance + repVariance

print cov.shape

# cov[:,:x.shape[0]] = np.tile(kbase.K(x), 4).T
# cov[:x.shape[0],:] = np.tile(kbase.K(x), 4)
# cov[:x.shape[0],:] = kbase.K(x)
cov[x.shape[0]:2*x.shape[0],x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance
cov[x.shape[0]:2*x.shape[0],3*x.shape[0]:] += kbatch.K(x) #- batchVarianceA
cov[3*x.shape[0]:,x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance

cov[2*x.shape[0]:3*x.shape[0],2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance
cov[2*x.shape[0]:3*x.shape[0],3*x.shape[0]:] += krep.K(x) #- repVariance
cov[3*x.shape[0]:,2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance

cov[3*x.shape[0]:,3*x.shape[0]:] += kbatch.K(x) + krep.K(x) # - repVariance-  batchVariance

# equal variance
# cov[range(cov.shape[0]),range(cov.shape[0])] = np.diag(cov).max()
sampleCov = cov.copy()

plt.imshow(sampleCov)

In [ ]:
s = generateSample(np.tile(f, 4), sampleCov, sigma)
y0, y2 , y1, y3 = np.array_split(s,4,1)

k0 = GPy.kern.RBF(1)
k1 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1)])
k2 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1)])
k3 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1), GPy.kern.RBF(1)])

y = y1

for z,k, kpred in [(x, k0, k0), (x1, k1, k1.rbf), (x2, k2, k2.rbf), (x3, k3, k3.rbf)]:
    m = GPy.models.GPRegression(z, y.T.reshape(50*ntot,1), k)
    m.randomize()
    m.optimize()

    mu,cov = m.predict_noiseless(z[:50,:],full_cov=True,kern=kpred)
    std = np.sqrt(cov.diagonal())
    diff = (mu[:,0] - f[:50])
    
    incorrect = 50-sum(((diff-1.98*std) < 0) & ((diff+1.98*std) > 0))
    
    print np.sum(np.linalg.eigvals(cov) > 1e-9)
    #print np.dot(diff, np.dot(np.linalg.inv(cov+np.eye(50)*cov.mean()), diff))
    print np.dot(diff, np.dot(np.linalg.inv(cov), diff))
    
    plt.figure()
    plt.subplot(121)
    plt.plot(x[:50,0],diff)
    plt.fill_between(x[:50,0], diff + 1.98*std, diff - 1.98*std, alpha=.1)
    
    plt.subplot(122)
    plt.plot(x[:50,0],mu[:,0])
    plt.fill_between(x[:50,0], mu[:,0] + 1.98*std, mu[:,0] - 1.98*std, alpha=.1)
    plt.plot(x[:50,0],f[:50])
    plt.plot(x[:50,0],y,c='k',lw=.1)

# m0 = GPy.models.GPRegression(x[:ntot*50,:], y0.T.reshape(50*ntot,1), k0)
# m0.randomize()
# m0.optimize()

# mu,cov = m0.predict_noiseless(x[:50,:],full_cov=True)
# diff = (mu[:,0] - f[:50])
# np.dot(diff, np.dot(np.linalg.inv(cov), diff))

In [ ]:
s = generateSample(np.tile(f, 4), sampleCov, sigma)
y0, y2 , y1, y3 = np.array_split(s,4,1)

for y in [y0, y1, y2, y3]:

    k0 = GPy.kern.RBF(1)
    k1 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1)])
    k2 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1)])
    k3 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1), GPy.kern.RBF(1)])

    for z,k, kpred in [(x, k0, k0), (x1, k1, k1.rbf), (x2, k2, k2.rbf), (x3, k3, k3.rbf)]:
        m = GPy.models.GPRegression(z, y.T.reshape(50*ntot,1), k)
        m.randomize()
        m.optimize()

        mu,cov = m.predict_noiseless(z[:50,:],full_cov=True,kern=kpred)
        std = np.sqrt(cov.diagonal())
        diff = (mu[:,0] - f[:50])

        incorrect = 50-sum(((diff-1.98*std) < 0) & ((diff+1.98*std) > 0))
        
        print incorrect

        #print np.sum(np.linalg.eigvals(cov) > 1e-9)
        #print np.dot(diff, np.dot(np.linalg.inv(cov+np.eye(50)*cov.mean()), diff))
        #print np.dot(diff, np.dot(np.linalg.inv(cov), diff))

        plt.figure()
        plt.subplot(121)
        plt.plot(x[:50,0],diff)
        plt.fill_between(x[:50,0], diff + 1.98*std, diff - 1.98*std, alpha=.1)

        plt.subplot(122)
        plt.plot(x[:50,0],mu[:,0])
        plt.fill_between(x[:50,0], mu[:,0] + 1.98*std, mu[:,0] - 1.98*std, alpha=.1)
        plt.plot(x[:50,0],f[:50])
        plt.plot(x[:50,0],y,c='k',lw=.1)
    
    print

In [ ]:
samples = []

In [ ]:
nsamp = 30

for _ in range(nsamp):
    samples.append([])

    s = generateSample(np.tile(f, 4), sampleCov, sigma)
    y0, y2 , y1, y3 = np.array_split(s,4,1)

    for y in [y0, y1, y2, y3]:
        
        samples[-1].append([])

        k0 = GPy.kern.RBF(1)
        k1 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1)])
        k2 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1)])
        k3 = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1), GPy.kern.RBF(1)])

        for z,k, kpred in [(x, k0, k0), (x1, k1, k1.rbf), (x2, k2, k2.rbf), (x3, k3, k3.rbf)]:
            m = GPy.models.GPRegression(z, y.T.reshape(nobs*ntot,1), k)
            m.randomize()
            m.optimize()

            mu,cov = m.predict_noiseless(z[:nobs,:],full_cov=True,kern=kpred)
            std = np.sqrt(cov.diagonal())
            diff = (mu[:,0] - f[:nobs])

            incorrect = nobs-sum(((diff-1.98*std) < 0) & ((diff+1.98*std) > 0))
            
            samples[-1][-1].append(incorrect)
            
            del m

In [ ]:
failrate = 1.*np.array(samples)/50

In [ ]:
# failrate

In [ ]:
# each figure is different model type
# each boxplot is generative type (m0-m3)

plt.figure()
plt.boxplot(np.array_split(failrate[:,:,0], 4, 1));

plt.figure()
plt.boxplot(np.array_split(failrate[:,:,1], 4, 1));

plt.figure()
plt.boxplot(np.array_split(failrate[:,:,2], 4, 1));

plt.figure()
plt.boxplot(np.array_split(failrate[:,:,3], 4, 1));

In [ ]:
plt.plot(x[:50,0],diff)
plt.fill_between(x[:50,0], diff + 1.98*std, diff - 1.98*std, alpha=.1)
# plt.plot(x[:50,0],f[:50])

In [ ]:
plt.plot(x[:50,0],mu[:,0])
plt.fill_between(x[:50,0], mu[:,0] + 1.98*std, mu[:,0] - 1.98*std, alpha=.1)
plt.plot(x[:50,0],f[:50])
plt.plot(x[:50,0],y2,c='k',lw=.4)

In [ ]:
mu,cov = m.predict_noiseless(x[:50,:],full_cov=True)
std = np.sqrt(cov.diagonal())

In [ ]:
sum(np.linalg.eigvals(cov) > 1e-9)

In [ ]:
diff = (mu[:,0] - f[:50])

np.dot(diff, np.dot(np.linalg.inv(cov), diff))

In [ ]:
diff.shape

In [ ]:
cov

In [ ]:
plt.plot(diff)
plt.plot(np.sqrt(cov));

In [ ]:
scipy.stats.chi2.ppf(.95, 50)

In [ ]:
scipy.stats.chi2.cdf(-2, 50)

In [ ]:
scipy.stats.chi2.cdf(np.dot(diff, np.dot(np.linalg.inv(cov), diff)), 50)

In [ ]:
std.shape, mu.shape

In [ ]:
plt.scatter(x[:ntot*50,0], y1.T.reshape(50*ntot,1))